# Lab | Agent & Vector store

**Change the state union dataset and replicate this lab by updating the prompts accordingly.**

One such dataset is the [sonnets.txt](https://github.com/martin-gorner/tensorflow-rnn-shakespeare/blob/master/shakespeare/sonnets.txt) dataset or any other data of your choice from the same git.

# Combine agents and vector stores

This notebook covers how to combine agents and vector stores. The use case for this is that you've ingested your data into a vector store and want to interact with it in an agentic manner.

The recommended method for doing so is to create a `RetrievalQA` and then use that as a tool in the overall agent. Let's take a look at doing this below. You can do this with multiple different vector DBs, and use the agent as a way to route between them. There are two different ways of doing this - you can either let the agent use the vector stores as normal tools, or you can set `return_direct=True` to really just use the agent as a router.

## Create the vector store

In [19]:
#!pip install chromadb

In [1]:
from langchain.chains import RetrievalQA
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAI, OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader

In [2]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')
HUGGINGFACEHUB_API_TOKEN = os.getenv('HUGGINGFACEHUB_API_TOKEN')

In [3]:
llm = OpenAI(temperature=0)

In [4]:
from pathlib import Path

relevant_parts = []
for p in Path(".").absolute().parts:
    relevant_parts.append(p)
    if relevant_parts[-3:] == ["langchain", "docs", "modules"]:
        break
doc_path = str(Path(*relevant_parts) / "state_of_the_union.txt")

In [5]:
loader = TextLoader(doc_path)
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings()
docsearch = Chroma.from_documents(texts, embeddings, collection_name="state-of-union")

In [6]:
state_of_union = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=docsearch.as_retriever()
)

In [7]:
from langchain_community.document_loaders import WebBaseLoader

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [8]:
loader = WebBaseLoader("https://beta.ruff.rs/docs/faq/")

In [9]:
docs = loader.load()
ruff_texts = text_splitter.split_documents(docs)
ruff_db = Chroma.from_documents(ruff_texts, embeddings, collection_name="ruff")
ruff = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=ruff_db.as_retriever()
)

Created a chunk of size 2134, which is longer than the specified 1000
Created a chunk of size 2470, which is longer than the specified 1000
Created a chunk of size 1259, which is longer than the specified 1000
Created a chunk of size 1053, which is longer than the specified 1000


## Create the Agent

In [10]:
# Import things that are needed generically
from langchain.agents import AgentType, Tool, initialize_agent
from langchain_openai import OpenAI

In [11]:
tools = [
    Tool(
        name="State of Union QA System",
        func=state_of_union.run,
        description="useful for when you need to answer questions about the most recent state of the union address. Input should be a fully formed question.",
    ),
    Tool(
        name="Ruff QA System",
        func=ruff.run,
        description="useful for when you need to answer questions about ruff (a python linter). Input should be a fully formed question.",
    ),
]

In [12]:
# Construct the agent. We will use the default agent type here.
# See documentation for a full list of options.
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

/Users/pedrorenan/.pyenv/versions/3.12.0/envs/ironhack/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.3.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [13]:
agent.invoke(
    "What did biden say about ketanji brown jackson in the state of the union address?"
)



> Entering new AgentExecutor chain...
 I should use the State of Union QA System to answer this question.
Action: State of Union QA System
Action Input: "What did Biden say about Ketanji Brown Jackson in the state of the union address?"
Observation:  Biden said that he nominated Ketanji Brown Jackson for the United States Supreme Court and praised her as one of the nation's top legal minds who will continue Justice Breyer's legacy of excellence.
Thought: I should use the Ruff QA System to get more information about Ketanji Brown Jackson.
Action: Ruff QA System
Action Input: "Who is Ketanji Brown Jackson?"
Observation:  I don't know.
Thought: I should use the State of Union QA System again to get more information about Ketanji Brown Jackson.
Action: State of Union QA System
Action Input: "Who is Ketanji Brown Jackson?"
Observation:  Ketanji Brown Jackson is a Circuit Court of Appeals Judge who has been nominated to serve on the United States Supreme Court by President Biden.
Thought: 

{'input': 'What did biden say about ketanji brown jackson in the state of the union address?',
 'output': 'Ketanji Brown Jackson is a Circuit Court of Appeals Judge who has been nominated to serve on the United States Supreme Court by President Biden.'}

In [14]:
agent.invoke("Why use ruff over flake8?")



> Entering new AgentExecutor chain...
 Ruff is a python linter that has some unique features compared to flake8, so it's worth exploring the differences.
Action: Ruff QA System
Action Input: "What are the unique features of ruff compared to flake8?"
Observation:  Ruff has over 800 implemented rules, compared to Flake8's ~409 rules. Ruff also has a different rule code and prefix system to minimize conflicts and allow for easier toggling of individual plugins. Additionally, Ruff does not support custom lint rules, but instead re-implements popular Flake8 plugins in Rust. Ruff also has some minor incompatibilities with Flake8 plugins, but many users have successfully switched from Flake8 to Ruff. Ruff also has the ability to automatically fix its own lint violations, unlike Flake8.
Thought: These unique features of Ruff make it a strong contender for a python linter.
Action: Ruff QA System
Action Input: "What are the benefits of using Ruff over Flake8?"
Observation:  Ruff offers a large

{'input': 'Why use ruff over flake8?',
 'output': 'Ruff offers a larger rule set, compatibility with Black, automatic fixing of lint violations, and support for Python 3.7 onwards, making it a strong contender for a python linter.'}

## Use the Agent solely as a router

You can also set `return_direct=True` if you intend to use the agent as a router and just want to directly return the result of the RetrievalQAChain.

Notice that in the above examples the agent did some extra work after querying the RetrievalQAChain. You can avoid that and just return the result directly.

In [15]:
tools = [
    Tool(
        name="State of Union QA System",
        func=state_of_union.run,
        description="useful for when you need to answer questions about the most recent state of the union address. Input should be a fully formed question.",
        return_direct=True,
    ),
    Tool(
        name="Ruff QA System",
        func=ruff.run,
        description="useful for when you need to answer questions about ruff (a python linter). Input should be a fully formed question.",
        return_direct=True,
    ),
]

In [16]:
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

In [17]:
agent.invoke(
    "What did biden say about ketanji brown jackson in the state of the union address?"
)



> Entering new AgentExecutor chain...
 I should use the State of Union QA System to answer this question.
Action: State of Union QA System
Action Input: "What did Biden say about Ketanji Brown Jackson in the state of the union address?"
Observation:  Biden said that he nominated Ketanji Brown Jackson for the United States Supreme Court and praised her as one of the nation's top legal minds who will continue Justice Breyer's legacy of excellence.


> Finished chain.


{'input': 'What did biden say about ketanji brown jackson in the state of the union address?',
 'output': " Biden said that he nominated Ketanji Brown Jackson for the United States Supreme Court and praised her as one of the nation's top legal minds who will continue Justice Breyer's legacy of excellence."}

In [18]:
agent.invoke("Why use ruff over flake8?")



> Entering new AgentExecutor chain...
 Ruff is a python linter that has some unique features compared to flake8, so it's worth exploring the differences.
Action: Ruff QA System
Action Input: "What are the unique features of ruff compared to flake8?"
Observation:  Ruff has over 800 implemented rules, compared to Flake8's ~409 rules. Ruff also has a different rule code and prefix system to minimize conflicts and allow for easier toggling of individual plugins. Additionally, Ruff does not support custom lint rules, but instead re-implements popular Flake8 plugins in Rust. Ruff also has some minor incompatibilities with Flake8 plugins, but many users have successfully switched from Flake8 to Ruff. Ruff also has the ability to automatically fix its own lint violations, unlike Flake8.


> Finished chain.


{'input': 'Why use ruff over flake8?',
 'output': " Ruff has over 800 implemented rules, compared to Flake8's ~409 rules. Ruff also has a different rule code and prefix system to minimize conflicts and allow for easier toggling of individual plugins. Additionally, Ruff does not support custom lint rules, but instead re-implements popular Flake8 plugins in Rust. Ruff also has some minor incompatibilities with Flake8 plugins, but many users have successfully switched from Flake8 to Ruff. Ruff also has the ability to automatically fix its own lint violations, unlike Flake8."}

## Multi-Hop vector store reasoning

Because vector stores are easily usable as tools in agents, it is easy to use answer multi-hop questions that depend on vector stores using the existing agent framework.

In [19]:
tools = [
    Tool(
        name="State of Union QA System",
        func=state_of_union.run,
        description="useful for when you need to answer questions about the most recent state of the union address. Input should be a fully formed question, not referencing any obscure pronouns from the conversation before.",
    ),
    Tool(
        name="Ruff QA System",
        func=ruff.run,
        description="useful for when you need to answer questions about ruff (a python linter). Input should be a fully formed question, not referencing any obscure pronouns from the conversation before.",
    ),
]

In [20]:
# Construct the agent. We will use the default agent type here.
# See documentation for a full list of options.
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

In [21]:
agent.invoke(
    "What tool does ruff use to run over Jupyter Notebooks? Did the president mention that tool in the state of the union?"
)



> Entering new AgentExecutor chain...
 I should check if the president mentioned the tool in the state of the union before answering the first question.
Action: State of Union QA System
Action Input: What tool did the president mention in the state of the union?
Observation:  The president mentioned the Bipartisan Infrastructure Law as a tool to rebuild America.
Thought: I should now check if ruff uses the Bipartisan Infrastructure Law to run over Jupyter Notebooks.
Action: Ruff QA System
Action Input: Does ruff use the Bipartisan Infrastructure Law to run over Jupyter Notebooks?
Observation:  No, Ruff does not use the Bipartisan Infrastructure Law to run over Jupyter Notebooks. It uses nbQA, a tool for running linters and code formatters over Jupyter Notebooks.
Thought: I now know the final answer.
Final Answer: No, Ruff does not use the Bipartisan Infrastructure Law to run over Jupyter Notebooks. It uses nbQA, a tool for running linters and code formatters over Jupyter Notebooks.



{'input': 'What tool does ruff use to run over Jupyter Notebooks? Did the president mention that tool in the state of the union?',
 'output': 'No, Ruff does not use the Bipartisan Infrastructure Law to run over Jupyter Notebooks. It uses nbQA, a tool for running linters and code formatters over Jupyter Notebooks.'}